In [ ]:
import numpy as np
from numpy.linalg import eig
import matplotlib.pyplot as plt

In [ ]:
def steering_vector(angle, nAntenna, error=False):
    d = np.arange(nAntenna)
    if error:
        return np.exp(-1j * np.pi * (d + np.random.randn()) * np.sin(angle))
    return np.exp(-1j * np.pi * d * np.sin(angle))

In [ ]:
def collect_planewave(angels, *args):
    steering_vec = np.empty((nAntenna, len(args)), dtype=np.complex64)
    amplitude_vec = np.empty((len(args), nSample), dtype=np.complex64)
    for i, signal in enumerate(args):
        steering_vec[:, i] = steering_vector(angels[i], nAntenna)
        signalPower = np.mean(np.abs(signal) ** 2)
        signal = signal * np.sqrt(10 ** (SNR * 0.1)) / np.sqrt(signalPower)
        amplitude_vec[i, :] = signal[0:nSample]

    planeWave = steering_vec @ amplitude_vec

    noise = np.random.randn(nAntenna, nSample) + 1j * np.random.randn(nAntenna, nSample)
    noise /= np.sqrt(np.var(noise))
    return planeWave + noise

In [ ]:
def root_music(input_signal, angle):
    #covariance matrix
    R = np.zeros((nAntenna, nAntenna), dtype=np.complex64)
    for i in range(nSample):
        R += np.outer(sig[:, i], np.conj(sig[:, i]).T)
    R /= nSample

    # calculate eigenvalues and eigenvectors of the correlate matrix
    eigenvalues, eigenvectors = eig(R)
    sorted_index = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalue = eigenvalues[sorted_index]
    sorted_eigenvectors = eigenvectors[:, sorted_index]

    # noise subspace
    Qn = sorted_eigenvectors[:, nSource:]

    v = steering_vector(angle, nAntenna)
    v_trans = np.conjugate(v).T

    J_z = v_trans.T @ Qn @ np.conjugate(Qn).T @ v.T
    return np.real(J_z)

In [ ]:
fcAntenna = 10e6
fs = 5000
t = np.arange(0, 1, 1/fs)
Lambda = 3e8 / fcAntenna
nAntenna = 16
nSample = 200
nSource = 1
SNR = 10

np.random.seed(42)

In [ ]:
angles = [np.pi/4, -np.pi/6]
smple_data1 = np.sin(2*np.pi*1000*t)
smple_data2 = np.sin(2*np.pi*1500*t)
sig = collect_planewave(angles, smple_data1, smple_data2)

In [ ]:
all_angles = np.arange(-np.pi/2, np.pi/2, 0.01)
result = np.zeros(len(all_angles))
for i, angle in enumerate(all_angles):
    result[i] = root_music(sig, angle)
plt.plot(all_angles, result)
plt.xlabel('degree')
plt.ylabel('DB')
plt.show()